In [1]:
import os
import pandas as pd
import torch
from dotenv import load_dotenv
from src.XRayDataset import XRayDataset
from src.utils import compute_mean_std, compute_class_weights, weighted_binary_crossentropy
from src.prePro import preprocess_metadata, calculate_balanced_label_statistics, stratified_split_by_individual_labels
from src.ICNTrainer import ICNTrainer
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn

# Load environment variables from .env file
load_dotenv()
data_dir = os.getenv('DATA_DIR')

filtered_df = preprocess_metadata(
    f'../{data_dir}/raw/xraysMD.csv',
    f'../{data_dir}/raw/xrays',
    f'../{data_dir}/processed/xraysMD.csv'
)

filtered_df_stats = calculate_balanced_label_statistics(filtered_df)



In [2]:
train_df, test_df = stratified_split_by_individual_labels(filtered_df, train_size=7000, test_size=3000)

# Print sizes of the resulting DataFrames
print(f"Training set size: {len(train_df)}")
print(f"Test/Validation set size: {len(test_df)}")

# Calculate the label distribution for the training and test sets
train_distribution = pd.Series([label for labels in train_df['Labels'] for label in labels]).value_counts()
test_distribution = pd.Series([label for labels in test_df['Labels'] for label in labels]).value_counts()

# Combine both distributions into a DataFrame
statistics_df = pd.DataFrame({
    'Training': train_distribution,
    'Test/Validation': test_distribution
})

# Fill NaN values with 0 (in case a label is not present in either set)
statistics_df.fillna(0, inplace=True)

# Add a total row to both columns
statistics_df.loc['Total'] = statistics_df.sum()

print("\nCombined label distribution statistics:")
print(statistics_df)


Training set size: 7000
Test/Validation set size: 3000

Combined label distribution statistics:
                    Training  Test/Validation
Atelectasis              674              284
Cardiomegaly             187              113
Consolidation            241              107
Edema                    100               40
Effusion                 652              295
Emphysema                138               59
Fibrosis                 142               65
Hernia                    17                6
Infiltration            1030              451
Mass                     294               97
No Finding              4092             1744
Nodule                   384              139
Pleural_Thickening       198               75
Pneumonia                 81               32
Pneumothorax             237              118
Total                   8467             3625


In [3]:
# Compute mean and std using a list of image paths
image_paths = [f"../data/raw/xrays/{image_id}.png" for image_id in train_df['ImageID']]
img_size = 1000  # Set to your desired size

mean, std = compute_mean_std(image_paths, img_size)

print(f"Computed Mean: {mean}, Computed Std: {std}")


Computed Mean: 139.45093172971428, Computed Std: 61.9332860849686


In [3]:
mean = 139.45
std = 61.93
img_size = 500
# Create training and validation datasets using the computed mean and std
train_dataset = XRayDataset(dataframe=train_df, image_dir='../data/raw/xrays/', img_size=img_size, mean=mean, std=std)
val_dataset = XRayDataset(dataframe=test_df, image_dir='../data/raw/xrays/', img_size=img_size, mean=mean, std=std)

# Create DataLoaders for batching and shuffling
batch_size = 5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [4]:
# Testing the DataLoader
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Print the shapes of the batch
print(f"Images batch shape: {images.shape}")  # Should be [batch_size, channels, height, width]
print(f"Labels batch shape: {labels.shape}")  # Should be [batch_size, num_labels]

# Check the individual data points (optional)
print(f"First image shape: {images[0].shape}")
print(f"First label: {labels[0]}")

Images batch shape: torch.Size([5, 1, 500, 500])
Labels batch shape: torch.Size([5, 15])
First image shape: torch.Size([1, 500, 500])
First label: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])


In [5]:
torch.cuda.empty_cache()
# Move y_true and y_pred to the same device as class_weights (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights = compute_class_weights(train_df, 15)
# Simulate a forward pass with some random inputs and test the loss function
y_true = torch.rand(5, 15).to(device)  # Same shape as your labels
y_pred = torch.rand(5, 15).to(device)  # Same shape as your model output
criterion = weighted_binary_crossentropy(class_weights)
# Now compute the loss value
loss_value = criterion(y_pred, y_true)
print(f"Loss value: {loss_value}")


Loss value: 1.618818998336792


In [7]:
# Load EfficientNet B3 model
model = models.efficientnet_b3(pretrained=True)

# Modify the input layer to accept 1-channel grayscale images (instead of the default 3-channel RGB)
model.features[0][0] = nn.Conv2d(1, model.features[0][0].out_channels, 
                                kernel_size=model.features[0][0].kernel_size, 
                                stride=model.features[0][0].stride, 
                                padding=model.features[0][0].padding, 
                                bias=False)

# Modify the output layer (classifier) to have 15 classes instead of 1000 (default for ImageNet)
num_classes = 15
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move the model to the device (GPU or CPU)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = ICNTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    project_name="inferno-calib-net",
    config={"learning_rate": 0.001, "batch_size": 16, "epochs": 5}
)

trainer.fit(epochs=5)


# # Define config dictionary with hyperparameters and metadata
# config = {
#     "learning_rate": 0.001,
#     "batch_size": 7,
#     "architecture": "EfficientNetB3",
#     "input_size": (1000, 1000),
#     "epochs": 5,
#     "optimizer": "Adam",
#     "loss_function": "Weighted BCE",
#     "class_weights": "Computed",
#     "dataset": "X-Ray Images",
#     "augmentation": "No",
# }

c:\Users\maksi\Desktop\master\InfernoCalibNet\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\maksi\Desktop\master\InfernoCalibNet\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5
--------------------


Training:   0%|          | 0/1400 [00:00<?, ?it/s]

Images shape: torch.Size([5, 1, 500, 500])
Labels shape: torch.Size([5, 15])
Outputs shape: torch.Size([5, 15])


Training:   0%|          | 1/1400 [00:28<11:08:54, 28.69s/it, loss=0.888]

...

Training:   5%|▍         | 69/1400 [07:49<1:16:58,  3.47s/it, loss=-57.2]

Images shape: torch.Size([5, 1, 500, 500])
Labels shape: torch.Size([5, 15])
Outputs shape: torch.Size([5, 15])


Training:   5%|▌         | 70/1400 [07:52<1:11:58,  3.25s/it, loss=-58]  

Images shape: torch.Size([5, 1, 500, 500])
Labels shape: torch.Size([5, 15])
Outputs shape: torch.Size([5, 15])


KeyboardInterrupt: 